# Predicting Drug's Reported Problems (Adverse Events) 

Andrew Chang, 2017/2/19

[The drug search app can be found here](https://ccchang0111.shinyapps.io/shinyapp/) (R Shiny app)

The purpose of this app is to find out **how likely a given drug or ingredient will cause problem** and allow user to compare similar drugs with the same active ingredients. 

When a person is sick, it might be hard to decide what medicine to take, because there are so many similar drugs under different brand-names that can treat the same symptom. Sometimes doctors might also find it difficult prescribing the right drug.  This app is potentially useful for anyone (**especially for those who do not have access to good healthcare**) when it comes to deciding what drugs to take, or to learn more about the drug they are currently taking. 

The "adverse event" in FDA's report can range from minor issues (like 'rash') to sever problems (like 'permanent deafness') and to wierd cases (like 'snake bite'?!). However, **not all the drugs on the market have well-documented adverse events (AE) in FDA's dataset**. Therefore, for those drugs without any AE reports, I apply **sklearn's CountVectorizer** and **SGDClassifier** to calculate their 'problematic level' based on their **name** and **active ingredients**. 

This document contains three sections:

### 1. <a href='#data_collection'>Data collection and cleaning (90% of the total work!!)</a>

### 2. <a href='#ft_label'> Generate Features and Lables (8%)</a>
   
### 3. <a href='#ML'>Training and Prediction</a>
(The last step only took 2% of the total work! Once the 'gound work' is well established, the rest is very easy)

In [1]:
import pandas as pd
import numpy as np
import pickle
from IPython.display import display
import sys
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

/Users/changc25/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


<a id='data_collection'></a>
# 1. Data collection and cleaning

Two important datasets used here:
- ### [Drugs and their active ingredients](https://www.fda.gov/Drugs/InformationOnDrugs/ucm079750.htm)
- ### [Drugs and their AE reports](http://www.nature.com/articles/sdata201626)

The purpose of this step is to create a cleaned table that looks like this:

[**Drug_Name, Ingredients, Average_AE_Odds_Ratio**]


## 1-1 Clean Drug Ingredient Data

In [2]:
filename1 = '.../Products.csv'
drug_info = pd.read_csv(filename1)
drug_info.head()

,ApplNo,ProductNo,Form,Strength,ReferenceDrug,DrugName,ActiveIngredient
0,4,4,SOLUTION/DROPS;OPHTHALMIC,1%,0,PAREDRINE,HYDROXYAMPHETAMINE HYDROBROMIDE
1,159,1,TABLET;ORAL,500MG,0,SULFAPYRIDINE,SULFAPYRIDINE
2,552,1,INJECTABLE;INJECTION,"20,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM
3,552,2,INJECTABLE;INJECTION,"40,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM
4,552,3,INJECTABLE;INJECTION,"5,000 UNITS/ML",0,LIQUAEMIN SODIUM,HEPARIN SODIUM


In [5]:
print 'number of unique drug names:',drug_info.DrugName.nunique()

number of unique drug names: 6850


Re-organize the above table into a dict:
** {Drug1 : [ActiveIngredient 1, ActiveIngredient 2, ...], Drug2: [ActiveIngredient 1, ...]..} **

In [117]:
# ignore drug with different dosing levels, because they all have the same ingredients
drug_dict = {}
for name in drug_info.DrugName.unique():
    ingredients = [x.lower() for x in drug_info.loc[drug_info.DrugName == name,'ActiveIngredient'].values[0].split(';')]
    drug_dict[name.lower()] = ingredients

In [113]:
drug_dict_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in drug_dict.iteritems() ])).T
drug_dict_df.reset_index(inplace=True)
drug_dict_df.rename(columns={'index': 'drug'}, inplace=True)

Remove '(' in the 'drug name' and 'first ingredient', because we are going to use these two columns to search AE reports in another dataset.

In [115]:
idx = drug_dict_df.drug.str.contains("\(")
drug_dict_df.loc[idx,'drug'] = drug_dict_df.loc[idx,'drug'].map(lambda drug : drug.split(' (')[0])

idx = drug_dict_df.loc[:,0].str.contains("\(")
drug_dict_df.loc[idx,0] = drug_dict_df.loc[idx,0].map(lambda drug : drug.split('(')[0])

In [117]:
drug_dict_df.head()

,drug,0,1,2,3,4,5,6,7,8,9,10,11,12
0,8-hour bayer,aspirin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8-mop,methoxsalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a-hydrocort,hydrocortisone sodium succinate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a-methapred,methylprednisolone sodium succinate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a-n stannous aggregated albumin,technetium tc-99m albumin aggregated kit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
drug_dict_df.to_csv('.../drug_dict_df.csv', index=None)

## 1-2 Clean Adverse Event Data

more than 500 Mb and more than 1M cases

In [50]:
lookup_col = ['ID', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lookup = pd.read_table('.../concept.tsv', names=lookup_col)

colname = ['drug_concept_id', 'outcome_concept_id', 'snomed_outcome_concept_id', 'case_count', 'prr', 'prr_95_upper','prr_95_lower', 'ror', 'ror_95_upper', 'ror_95_lower']
AE = pd.read_table('.../standard_drug_outcome_statistics.tsv', names = colname)

lookup.loc[:,1] = lookup.loc[:,1].str.lower()

In [51]:
lookup.head() # names for all the concept_id in AE data

,ID,1,2,3,4,5,6,7,8,9,10
0,0,no matching concept,Metadata,None,Undefined,NaN,No matching concept,1970-01-01,2099-12-31,NaN,NaN
1,1,domain,Metadata,Domain,Domain,NaN,OMOP generated,1970-01-01,2099-12-31,NaN,NaN
2,2,gender,Metadata,Domain,Domain,NaN,OMOP generated,1970-01-01,2099-12-31,NaN,NaN
3,3,race,Metadata,Domain,Domain,NaN,OMOP generated,1970-01-01,2099-12-31,NaN,NaN
4,4,ethnicity,Metadata,Domain,Domain,NaN,OMOP generated,1970-01-01,2099-12-31,NaN,NaN


In [52]:
AE.head()

,drug_concept_id,outcome_concept_id,snomed_outcome_concept_id,case_count,prr,prr_95_upper,prr_95_lower,ror,ror_95_upper,ror_95_lower
0,918906,35104067,432588,1,1.10027,7.82057,0.15480,1.10027,7.82080,0.15479
1,19080406,35104070,436659,1,2.60101,18.45177,0.36665,2.60230,18.49021,0.36625
2,1115572,35104074,439777,108,0.53708,0.64839,0.44488,0.53570,0.64709,0.44348
3,1315027,35104074,439777,12,0.53124,0.93466,0.30194,0.52985,0.93379,0.30065
4,43526424,35104074,439777,65,0.12473,0.15905,0.09782,0.12412,0.15830,0.09733


## 1-3 Remove unreliable AE data

### Collect only True Positive AE (OR 95%CI lower > 1) and True Negative AE (OR 95%CI upper < 1)

In [101]:
# remove those that do not contain Odds Ratios, mostly single digit case_count
idx = AE.ror.str.contains('\N')
AE2 = AE[~idx.fillna(True)].loc[:,['drug_concept_id', 'outcome_concept_id', 'case_count', 'ror', 'ror_95_upper', 'ror_95_lower']]
display(AE2.head())
print "%d of data are removed." %(AE.shape[0] - AE2.shape[0])

,drug_concept_id,outcome_concept_id,case_count,ror,ror_95_upper,ror_95_lower
0,918906,35104067,1,1.10027,7.82080,0.15479
1,19080406,35104070,1,2.60230,18.49021,0.36625
2,1115572,35104074,108,0.53570,0.64709,0.44348
3,1315027,35104074,12,0.52985,0.93379,0.30065
4,43526424,35104074,65,0.12412,0.15830,0.09733


66332 of data are removed.


In [103]:
# convert ror to float
AE2.loc[:,['ror','ror_95_upper','ror_95_lower']] = AE2.loc[:,['ror','ror_95_upper','ror_95_lower']].astype(float)

In [107]:
idx_p = AE2.ror_95_lower >= 1
AE_p = AE2[idx_p]

idx_n = AE2.ror_95_upper < 1
AE_n = AE2[idx_n]

In [110]:
display(AE_p.head())
display(AE_n.head())
print "OD >=1: ", AE_p.shape[0]
print "OD < 1: ", AE_n.shape[0]

,drug_concept_id,outcome_concept_id,case_count,ror,ror_95_upper,ror_95_lower
5,501343,35104078,1,63.74335,454.98472,8.93044
6,529411,35104078,1,15.46275,109.93527,2.17489
7,529660,35104078,1,16.43847,116.88161,2.31194
8,700253,35104078,2,4.32361,17.29720,1.08073
12,704943,35104078,7,2.95493,6.20212,1.40785


,drug_concept_id,outcome_concept_id,case_count,ror,ror_95_upper,ror_95_lower
2,1115572,35104074,108,0.53570,0.64709,0.44348
3,1315027,35104074,12,0.52985,0.93379,0.30065
4,43526424,35104074,65,0.12412,0.15830,0.09733
13,705103,35104078,3,0.19179,0.59488,0.06183
14,705944,35104078,1,0.12864,0.91344,0.01812


OD >=1:  1030714
OD < 1:  287370


## 1-4 Map 'Drug Names' and 'Outcome Names' into above tables

In [96]:
def drugname_lookup(ID):
    tmp = lookup.loc[lookup.ID == ID, 1].values
    if tmp.size > 0:
        drugname = tmp[0]
    else :
        drugname = np.nan
    return drugname

In [64]:
AE_n['drug_name'] = AE_n.drug_concept_id.apply(drugname_lookup)

In [97]:
AE_n['outcome_name'] = AE_n.outcome_concept_id.apply(drugname_lookup)

In [98]:
print "%d of AE_n in total" %(AE_n.shape[0])
print "%d of AE_n NA in drug name" %(sum(AE_n.drug_name.isnull()))
AE_n.dropna(inplace=True) # drop all the NA
print "%d of AE_n after dropping NAs" %(AE_n.shape[0])

287315 of AE_n in total
0 of AE_n NA in drug name
287315 of AE_n after dropping NAs


In [68]:
AE_p['drug_name'] = AE_p.drug_concept_id.apply(drugname_lookup)

In [100]:
AE_p['outcome_name'] = AE_p.outcome_concept_id.apply(drugname_lookup)

In [92]:
print "%d of AE_p in total" %(AE_p.shape[0])
print "%d of AE_p NA in drug name" %(sum(AE_p.drug_name.isnull()))
AE_p.dropna(inplace=True)
print "%d of AE_p after dropping NAs" %(AE_p.shape[0])

1030714 of AE_p in total
3500 of AE_p NA in drug name
1027214 of AE_p after dropping NAs


In [101]:
display(AE_p.head())
display(AE_n.head())

,drug_concept_id,outcome_concept_id,case_count,ror,ror_95_upper,ror_95_lower,drug_name,outcome_name
5,501343,35104078,1,63.74335,454.98472,8.93044,hepatitis b immune globulin,anaemia postoperative
6,529411,35104078,1,15.46275,109.93527,2.17489,"tetanus toxoid vaccine, inactivated",anaemia postoperative
7,529660,35104078,1,16.43847,116.88161,2.31194,hepatitis a vaccine (inactivated) strain hm175,anaemia postoperative
8,700253,35104078,2,4.32361,17.29720,1.08073,thiopental,anaemia postoperative
12,704943,35104078,7,2.95493,6.20212,1.40785,methocarbamol,anaemia postoperative


,drug_concept_id,outcome_concept_id,case_count,ror,ror_95_upper,ror_95_lower,drug_name,outcome_name
2,1115572,35104074,108,0.53570,0.64709,0.44348,beclomethasone,anaemia
3,1315027,35104074,12,0.52985,0.93379,0.30065,cranberry preparation,anaemia
4,43526424,35104074,65,0.12412,0.15830,0.09733,dimethyl fumarate,anaemia
13,705103,35104078,3,0.19179,0.59488,0.06183,lamotrigine,anaemia postoperative
14,705944,35104078,1,0.12864,0.91344,0.01812,methylphenidate,anaemia postoperative


In [105]:
AE_n.to_csv('.../AE_n.csv', index=None)
AE_p.to_csv('.../AE_p.csv', index=None)
AE_p.append(AE_n).to_csv('.../AE_all.csv', index=None)

## 1-5 Calculate 'Average' Odds Ratio for each drug that has ingredient info

For the following table, we are going to add few more columns including **'average' Odds Ratio**, which will be used to calculate **'problematic level'**

In [33]:
drug_dict_df.head()

,drug,0,1,2,3,4,5,6,7,8,9,10,11,12
0,8-hour bayer,aspirin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8-mop,methoxsalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,a-hydrocort,hydrocortisone sodium succinate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a-methapred,methylprednisolone sodium succinate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a-n stannous aggregated albumin,technetium tc-99m albumin aggregated kit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To calculate the average score, for a given drug name, we find ALL the AE cases reported under that drug name, calculate basic statistics. If drug name search does not find any, use the drug's 1st ingredient and search again. This is because **some AE cases use generic drug name rather than the brand name**.  

In [162]:
# next time starts from n = 1409
drug_stats = {}
counter = 0
length = len(drug_dict.keys())
for i, name in enumerate(drug_dict_df.drug):   
    counter += 1
    drug_name = name    
    idx_p = AE_p.drug_name.str.contains(name)
    idx_n = AE_n.drug_name.str.contains(name)

    num_p = sum(idx_p) # number of pos reports
    num_n = sum(idx_n) # number of neg reports

    if (num_p == 0) & (num_n == 0): # if the drugname does not match the name in AE, use the drug-ingredient
        name = drug_dict_df.loc[i,0]

        idx_p = AE_p.drug_name.str.contains(name)
        idx_n = AE_n.drug_name.str.contains(name)

        num_p = sum(idx_p) # number of pos reports
        num_n = sum(idx_n) # number of neg reports
    
    tot_OD_p = AE_p.loc[idx_p,'ror'].sum()
    tot_OD_n = AE_n.loc[idx_n,'ror'].sum()
    avg_OD_p = AE_p.loc[idx_p,'ror'].mean()
    avg_OD_n = AE_n.loc[idx_n,'ror'].mean()
    
    drug_stats[drug_name] = [num_p, num_n, tot_OD_p, tot_OD_n, avg_OD_p, avg_OD_n]
    
    sys.stdout.write('\r %d out of %d' %(counter, length))
    sys.stdout.flush()

 6848 out of 6848

In [163]:
drug_stats_1 = pd.DataFrame(drug_stats).T.reset_index()
drug_stats_1.columns = ['drug',' num_p', 'num_n', 'tot_OD_p', 'tot_OD_n', 'avg_OD_p', 'avg_OD_n']
print drug_stats_1.shape
display(drug_stats_1.head())

(6811, 7)


,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n
0,8-hour bayer,2645.0,1685.0,234588.97006,858.09978,88.691482,0.509258
1,8-mop,189.0,0.0,28518.76649,0.00000,150.892944,NaN
2,a-hydrocort,0.0,0.0,0.00000,0.00000,NaN,NaN
3,a-methapred,0.0,0.0,0.00000,0.00000,NaN,NaN
4,a-n stannous aggregated albumin,0.0,0.0,0.00000,0.00000,NaN,NaN


You can see some drugs do not have Odds Ratio (NaN), those are the ones that we are going to predict later.

In [164]:
drug_stats_1.to_csv('.../drug_stats.csv', index=None)

<a id='merged_AE_Drug_info'></a>

### Now we have achieved our 1st goal, making a table that contains drug name, ingredients, average odds ratio.

In [182]:
drug_summary.sort_values('num_n', ascending=False).head()

,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,3,4,5,6,7,8,9,10,11,12
2965,insulin,7170.0,3890.0,215789.07410,1733.80871,30.096105,0.445709,insulin pork,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2184,estrace,5491.0,3000.0,762854.43663,772.28900,138.928144,0.257430,estradiol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3805,minivelle,5491.0,3000.0,762854.43663,772.28900,138.928144,0.257430,estradiol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4972,prefest,5491.0,3000.0,762854.43663,772.28900,138.928144,0.257430,estradiol,norgestimate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024,elestrin,5491.0,3000.0,762854.43663,772.28900,138.928144,0.257430,estradiol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='ft_label'></a>
# 2. Generate Features and Lables

## 2-1 Separate Data for Training and Data for Prediction

In [189]:
idx = (drug_summary.avg_OD_p > 0) | (drug_summary.avg_OD_n > 0)
idx_pred = ~idx

drug_summary_learn = drug_summary[idx]
drug_summary_pred = drug_summary[idx_pred]

In [193]:
print "%d data in Total" %(drug_summary.shape[0])
print "%d data in learning set" %(drug_summary_learn.shape[0])
drug_summary_learn.head()

6848 data in Total
3645 data in learning set


,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,3,4,5,6,7,8,9,10,11,12
0,8-hour bayer,2645.0,1685.0,234588.97006,858.09978,88.691482,0.509258,aspirin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8-mop,189.0,0.0,28518.76649,0.00000,150.892944,NaN,methoxsalen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,a/t/s,1142.0,167.0,73239.58092,76.67688,64.132733,0.459143,erythromycin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,abacavir,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir sulfate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,abacavir and lamivudine,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir,lamivudine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [192]:
print "%d data in predicting set" %(drug_summary_pred.shape[0])
drug_summary_pred.head()

3203 data in predicting set


,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,3,4,5,6,7,8,9,10,11,12
2,a-hydrocort,0.0,0.0,0.0,0.0,NaN,NaN,hydrocortisone sodium succinate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,a-methapred,0.0,0.0,0.0,0.0,NaN,NaN,methylprednisolone sodium succinate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,a-n stannous aggregated albumin,0.0,0.0,0.0,0.0,NaN,NaN,technetium tc-99m albumin aggregated kit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,a-poxide,0.0,0.0,0.0,0.0,NaN,NaN,chlordiazepoxide hydrochloride,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,a.p.l.,0.0,0.0,0.0,0.0,NaN,NaN,"gonadotropin, chorionic",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have decent amount of data for trainig (3645 cases)

<a id='avg_odd'></a>
## 2-2 Calculate "Ultimate Odds Ratio" for further classification

Odds Ratio = 1 means that reported AE is insignificant. If OR >> 1, the AE case is significant; if OR < 1 means it is 'less likely' to happen (or it might even be preventive).

Some drugs have both cases with OR > 1 and OR < 1, for example "8-hour bayer" has average OR= 88.7 for significant cases and average OR = 0.5 for 'preventive' cases. To calculate the **ultimate odds ratio**, we can **multiply value of OR > 1 and value of OR < 1**. 

My idea behind this is, for example, there are two drugs with same values of OR > 1, both OR = 100. But Drug A has other cases with OR = 0.001 and Drug B has other cases with OR = 0.8. So overall Drug A should be less problematic, and this can be calculated by comparing 100*0.001 vs 100*0.8. 

Why not just 'add them'? Becuase the dynamic range for OR > 1 and OR < 1 is very different, adding OR < 1 values can be very ineffective for those OR >>> 1. Therefore multiplication is better. And to avoid OR>1 * 0 that make the whole case 0, we need to add '1' to all the OR < 1 cases; this is to 'shift' baseline to 1. For example, a case with OR = 100 and OR = 0, the multiplication gives 0, that does not mean the drug has no AE, it only means the drug might not be that problematic as those severe cases. (However, you don't really need to do +1 shfit. This is just my preference.)

In [203]:
# compute an 'ultimate OR' = avg_OD_p * avg_OD_n
# we need to avg_OD_n + 1
# we need to avg_OD_p + 0 (especially for NAs) 
# this will make the multiply product 0, if OD_p is NA->0
drug_summary_learn.loc[:,'avg_OD_n_new'] = drug_summary_learn.avg_OD_n.fillna(0) + 1
drug_summary_learn.loc[:,'avg_OD_p_new'] = drug_summary_learn.avg_OD_p.fillna(0)
drug_summary_learn.loc[:,'alt_OR'] = drug_summary_learn.avg_OD_n_new * drug_summary_learn.avg_OD_p_new 
drug_summary_learn.head()

/Users/changc25/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,...,6,7,8,9,10,11,12,avg_OD_n_new,avg_OD_p_new,alt_OR
0,8-hour bayer,2645.0,1685.0,234588.97006,858.09978,88.691482,0.509258,aspirin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.509258,88.691482,133.858331
1,8-mop,189.0,0.0,28518.76649,0.00000,150.892944,NaN,methoxsalen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,150.892944,150.892944
7,a/t/s,1142.0,167.0,73239.58092,76.67688,64.132733,0.459143,erythromycin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.459143,64.132733,93.578828
8,abacavir,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir sulfate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.307948,44.410783,58.087014
9,abacavir and lamivudine,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir,lamivudine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.307948,44.410783,58.087014


## 2-3 Define Drug's Problem Levels

We are going to define 4 levels: 
- **0 is 'probably no problem'**
- **1 is 'may have some problems'**
- **2 is 'there are some problems'**
- **3 is 'be careful'**

In [232]:
print "total data", drug_summary_learn.alt_OR.count()
print "max value", drug_summary_learn.alt_OR.max()
print "min value", drug_summary_learn.alt_OR.min()

total data 3645
max value 2525328.99805
min value 6.85863095506


The reason I chose those ranges is because they make good partition among each level, so we don't have to worry too much about dealing with imbalanced data.

In [261]:
idx0 = (drug_summary_learn.alt_OR < 40) & (drug_summary_learn.alt_OR > 1)
idx1 = (drug_summary_learn.alt_OR < 80) & (drug_summary_learn.alt_OR >= 40)
idx2 = (drug_summary_learn.alt_OR < 160) & (drug_summary_learn.alt_OR >= 80)
idx3 = (drug_summary_learn.alt_OR >= 160)

print "Num of label Fine 0    :", sum(idx0)
print "Num of label OK 1      :", sum(idx1)
print "Num of label Bad 2     :", sum(idx2)
print "Num of label Very Bad 3:", sum(idx3)

Num of label Fine 0    : 893
Num of label OK 1      : 909
Num of label Bad 2     : 908
Num of label Very Bad 3: 935


In [265]:
# create labels
drug_summary_learn['Label'] = 0
drug_summary_learn.loc[idx1,'Label'] = 1
drug_summary_learn.loc[idx2,'Label'] = 2
drug_summary_learn.loc[idx3,'Label'] = 3

drug_summary_learn.head()

,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,...,7,8,9,10,11,12,avg_OD_n_new,avg_OD_p_new,alt_OR,Label
0,8-hour bayer,2645.0,1685.0,234588.97006,858.09978,88.691482,0.509258,aspirin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.509258,88.691482,133.858331,2
1,8-mop,189.0,0.0,28518.76649,0.00000,150.892944,NaN,methoxsalen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,150.892944,150.892944,2
7,a/t/s,1142.0,167.0,73239.58092,76.67688,64.132733,0.459143,erythromycin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.459143,64.132733,93.578828,2
8,abacavir,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir sulfate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.307948,44.410783,58.087014,1
9,abacavir and lamivudine,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir,lamivudine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.307948,44.410783,58.087014,1


## 2-4 Combine Drug Name and their Ingredients

because these are going to be the features for machine learning

In [289]:
drug_comp = drug_summary_learn.loc[:,['drug',0,1,2,3,4,5,6,7,8,9,10,11,12]]
drug_comp = drug_comp.fillna('')
drug_comp.head(10)

,drug,0,1,2,3,4,5,6,7,8,9,10,11,12
0,8-hour bayer,aspirin,,,,,,,,,,,,
1,8-mop,methoxsalen,,,,,,,,,,,,
7,a/t/s,erythromycin,,,,,,,,,,,,
8,abacavir,abacavir sulfate,,,,,,,,,,,,
9,abacavir and lamivudine,abacavir,lamivudine,,,,,,,,,,,
14,abacavir; lamivudine,abacavir,lamivudine,,,,,,,,,,,
15,abelcet,amphotericin b,,,,,,,,,,,,
16,abilify,aripiprazole,,,,,,,,,,,,
17,abilify maintena kit,aripiprazole,,,,,,,,,,,,
20,abraxane,paclitaxel,,,,,,,,,,,,


In [288]:
drug_comp_full = drug_comp.drug 
for i in range(13):
    drug_comp_full = drug_comp_full + ' ' + drug_comp.loc[:,i]
        
drug_comp_full.head(10)

0                      8-hour bayer aspirin            
1                         8-mop methoxsalen            
7                        a/t/s erythromycin            
8                 abacavir abacavir sulfate            
9     abacavir and lamivudine abacavir lamivudine   ...
14    abacavir; lamivudine abacavir lamivudine      ...
15                   abelcet amphotericin b            
16                     abilify aripiprazole            
17        abilify maintena kit aripiprazole            
20                      abraxane paclitaxel            
dtype: object

In [296]:
data_learn = pd.DataFrame({'Drug': drug_comp_full, 'Label':drug_summary_learn.Label})
print data_learn.shape
display(data_learn.head())

(3645, 2)


,Drug,Label
0,8-hour bayer aspirin,2
1,8-mop methoxsalen,2
7,a/t/s erythromycin,2
8,abacavir abacavir sulfate,1
9,abacavir and lamivudine abacavir lamivudine ...,1


In [300]:
# dealing with the prediction data
drug_comp_pred = drug_summary_pred.loc[:,['drug',0,1,2,3,4,5,6,7,8,9,10,11,12]]
drug_comp_pred = drug_comp_pred.fillna('')
drug_comp_pred.head(10)

,drug,0,1,2,3,4,5,6,7,8,9,10,11,12
2,a-hydrocort,hydrocortisone sodium succinate,,,,,,,,,,,,
3,a-methapred,methylprednisolone sodium succinate,,,,,,,,,,,,
4,a-n stannous aggregated albumin,technetium tc-99m albumin aggregated kit,,,,,,,,,,,,
5,a-poxide,chlordiazepoxide hydrochloride,,,,,,,,,,,,
6,a.p.l.,"gonadotropin, chorionic",,,,,,,,,,,,
10,abacavir sulfate,abacavir sulfate,,,,,,,,,,,,
11,abacavir sulfate and lamivudine,abacavir sulfate,lamivudine,,,,,,,,,,,
12,"abacavir sulfate, lamivudine and zidovudine",abacavir sulfate,lamivudine,zidovudine,,,,,,,,,,
13,abacavir sulfate; lamivudine,abacavir sulfate,lamivudine,,,,,,,,,,,
18,abitrexate,methotrexate sodium,,,,,,,,,,,,


In [299]:
drug_comp_pred_full = drug_comp_pred.drug 
for i in range(13):
    drug_comp_pred_full = drug_comp_pred_full + ' ' + drug_comp_pred.loc[:,i]
        
drug_comp_pred_full.head(10)

2     a-hydrocort hydrocortisone sodium succinate   ...
3     a-methapred methylprednisolone sodium succinat...
4     a-n stannous aggregated albumin technetium tc-...
5     a-poxide chlordiazepoxide hydrochloride       ...
6            a.p.l. gonadotropin, chorionic            
10        abacavir sulfate abacavir sulfate            
11    abacavir sulfate and lamivudine abacavir sulfa...
12    abacavir sulfate, lamivudine and zidovudine ab...
13    abacavir sulfate; lamivudine abacavir sulfate ...
18           abitrexate methotrexate sodium            
dtype: object

In [302]:
data_pred = drug_comp_pred_full
data_learn.to_csv('.../data_learn.csv', index=None)
data_pred.to_csv('.../data_pred.csv', index=None)

In [2]:
data_learn = pd.read_csv('.../data_learn.csv')
data_pred = pd.read_csv('.../data_pred.csv')

## 2-5 Create Features using CountVectorizer (Bag-of-Words)

In [3]:
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

X_train, X_test, y_train, y_test = train_test_split(data_learn.Drug, data_learn.Label, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2733,)
(912,)
(2733,)
(912,)


/Users/changc25/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_train_dtm

<2733x2827 sparse matrix of type '<type 'numpy.int64'>'
	with 10479 stored elements in Compressed Sparse Row format>

In [5]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

<912x2827 sparse matrix of type '<type 'numpy.int64'>'
	with 2712 stored elements in Compressed Sparse Row format>

<a id='ML'></a>
# 3. Training and Prediction 

## 3-1 Try MultinomialNB

In [309]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [310]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
%time nb.fit(X_train_dtm, y_train)

CPU times: user 3.47 ms, sys: 2.33 ms, total: 5.8 ms
Wall time: 13.7 ms


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [312]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print "accuracy:", metrics.accuracy_score(y_test, y_pred_class)
print metrics.classification_report(y_test, y_pred_class)

accuracy: 0.794956140351
             precision    recall  f1-score   support

          0       0.89      0.83      0.86       216
          1       0.91      0.73      0.81       232
          2       0.83      0.79      0.81       234
          3       0.63      0.83      0.72       230

avg / total       0.82      0.79      0.80       912



## 3-2 Try SGDClassifier
Linear classifiers (SVM, logistic regression, a.o.) with SGD training.

In [15]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV, validation_curve, StratifiedShuffleSplit

In [29]:
clf = SGDClassifier()
%time clf.fit(X_train_dtm, y_train)

CPU times: user 10.8 ms, sys: 859 µs, total: 11.6 ms
Wall time: 10.9 ms


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [38]:
cv_sets = StratifiedShuffleSplit(n_splits=3, test_size=0.2, random_state=1)
parameters = {'alpha': [0.01, 0.001, 0.0001], 'penalty': ['l1', 'l2', 'elasticnet']}

grid = GridSearchCV(clf, parameters, cv=cv_sets, scoring = 'accuracy', n_jobs=3)
%time grid.fit(X_train_dtm, y_train)
print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))
print grid.best_estimator_

CPU times: user 162 ms, sys: 42.6 ms, total: 205 ms
Wall time: 409 ms
The best parameters are {'penalty': 'l2', 'alpha': 0.001} with a score of 0.84
SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)


In [45]:
clf = grid.best_estimator_
%time clf.fit(X_train_dtm, y_train)

CPU times: user 7.79 ms, sys: 702 µs, total: 8.49 ms
Wall time: 7.77 ms


SGDClassifier(alpha=0.001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [44]:
# make class predictions for X_test_dtm
y_pred_class = clf.predict(X_test_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print "accuracy:", metrics.accuracy_score(y_test, y_pred_class)
print metrics.classification_report(y_test, y_pred_class)

accuracy: 0.8125
             precision    recall  f1-score   support

          0       0.62      0.93      0.75       216
          1       0.92      0.81      0.86       232
          2       0.93      0.82      0.87       234
          3       0.89      0.70      0.78       230

avg / total       0.85      0.81      0.82       912



- precision: The precision is intuitively the ability of the classifier not to label as positive a sample that is negative
- recall: The recall is intuitively the ability of the classifier to find all the positive samples.
- f1-score: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0.

**Seems like SGDClassifier does better than Naive Bayes without any tuning**

In [332]:
X_test.head(10)

5631    sodium pertechnetate tc 99m technetium tc-99m ...
1135            chloromycetin chloramphenicol            
3886                       mozobil plerixafor            
1621            desvenlafaxine desvenlafaxine            
1092                     cetrotide cetrorelix            
3676    metatensin #2 reserpine  trichlormethiazide   ...
5344                      reserpine reserpine            
5379                        ridaura auranofin            
3290                    leukeran chlorambucil            
6202    travasol 4.25% sulfite free w/ electrolytes in...
Name: Drug, dtype: object

In [331]:
print "actual label:", (y_test[:20].values)
print "predicted   :", (y_pred_class[:20])

actual label: [3 3 2 0 3 3 3 3 2 1 2 1 3 2 1 3 1 1 0 3]
predicted   : [3 3 3 0 3 3 3 3 3 1 2 1 3 2 1 3 1 1 0 3]


## 3-3 Predicting those drugs without AE reports

In [333]:
data_pred.head()

2    a-hydrocort hydrocortisone sodium succinate   ...
3    a-methapred methylprednisolone sodium succinat...
4    a-n stannous aggregated albumin technetium tc-...
5    a-poxide chlordiazepoxide hydrochloride       ...
6           a.p.l. gonadotropin, chorionic            
dtype: object

In [334]:
data_pred_dtm = vect.transform(data_pred)
data_pred_dtm

<3203x2827 sparse matrix of type '<type 'numpy.int64'>'
	with 6234 stored elements in Compressed Sparse Row format>

In [341]:
data_pred_label = clf.predict(data_pred_dtm)
drug_summary_pred['pred_label'] = data_pred_label

/Users/changc25/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [343]:
print "Data with AE reports"
display(drug_summary_learn.head())
print "Data without AE reports but with predicted scores"
display(drug_summary_pred.head())

Data with AE reports


,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,...,7,8,9,10,11,12,avg_OD_n_new,avg_OD_p_new,alt_OR,Label
0,8-hour bayer,2645.0,1685.0,234588.97006,858.09978,88.691482,0.509258,aspirin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.509258,88.691482,133.858331,2
1,8-mop,189.0,0.0,28518.76649,0.00000,150.892944,NaN,methoxsalen,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,150.892944,150.892944,2
7,a/t/s,1142.0,167.0,73239.58092,76.67688,64.132733,0.459143,erythromycin,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.459143,64.132733,93.578828,2
8,abacavir,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir sulfate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.307948,44.410783,58.087014,1
9,abacavir and lamivudine,1724.0,357.0,76564.19000,109.93759,44.410783,0.307948,abacavir,lamivudine,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.307948,44.410783,58.087014,1


Data without AE reports but with predicted scores


,drug,num_p,num_n,tot_OD_p,tot_OD_n,avg_OD_p,avg_OD_n,0,1,2,...,4,5,6,7,8,9,10,11,12,pred_label
2,a-hydrocort,0.0,0.0,0.0,0.0,NaN,NaN,hydrocortisone sodium succinate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,a-methapred,0.0,0.0,0.0,0.0,NaN,NaN,methylprednisolone sodium succinate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,a-n stannous aggregated albumin,0.0,0.0,0.0,0.0,NaN,NaN,technetium tc-99m albumin aggregated kit,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5,a-poxide,0.0,0.0,0.0,0.0,NaN,NaN,chlordiazepoxide hydrochloride,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,a.p.l.,0.0,0.0,0.0,0.0,NaN,NaN,"gonadotropin, chorionic",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3


In [344]:
drug_summary_learn.to_csv('.../drug_summary_learn.csv', index=None)
drug_summary_pred.to_csv('.../drug_summary_pred.csv', index=None)